In [1]:
import pandas as pd

In [5]:
table=pd.read_csv('/home/ls/rachelcw/projects/SPOT/spot/example_data/exon_exon_junction_file.txt', sep=' ')

sample_0\tsample_1\tsample_2\tsample_3\tsample_4\tsample_5\tsample_6\tsample_7\tsample_8\tsample_9\tsample_10\tsample_11\tsample_12\tsample_13\tsample_14\tsample_15\tsample_16\tsample_17\tsample_18\tsample_19\tsample_20\tsample_21\tsample_22\tsample_23\tsample_24\tsample_25\tsample_26\tsample_27\tsample_28\tsample_29\tsample_30\tsample_31\tsample_32\tsample_33\tsample_34\tsample_35\tsample_36\tsample_37\tsample_38\tsample_39\tsample_40\tsample_41\tsample_42\tsample_43\tsample_44\tsample_45\tsample_46\tsample_47\tsample_48\tsample_49\tsample_50\tsample_51\tsample_52\tsample_53\tsample_54\tsample_55\tsample_56\tsample_57\tsample_58\tsample_59\tsample_60\tsample_61\tsample_62\tsample_63\tsample_64\tsample_65\tsample_66\tsample_67\tsample_68\tsample_69\tsample_70\tsample_71\tsample_72\tsample_73\tsample_74\tsample_75\tsample_76\tsample_77\tsample_78\tsample_79\tsample_80\tsample_81\tsample_82\tsample_83\tsample_84\tsample_85\tsample_86\tsample_87\tsample_88\tsample_89\tsample_90\tsample_91

In [6]:
# create list of blood samples from gtex #

# annot=pd.read_csv("/data01/private/projects/splicing_rd/GTEx/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt",sep="\t")
# annot_blood=annot.loc[annot["SMTSD"]=="Whole Blood"]

# annot_blood["SAMPID"].to_csv("~/projects/SPOT/sample_blood_gtex.20230319.txt",sep="\t", index=0)

Intergration leafcutter results (next step with spot)

In [2]:
# # cluster_significance # #
cluster=pd.read_csv("/home/ls/rachelcw/projects/rare_disease/data/pilot1_ds_cluster_significance.txt",sep="\t",usecols=["cluster","p.adjust"])
cluster[["chrom","clu"]]=cluster['cluster'].str.split(':', expand=True)
cluster.drop(['cluster'], axis=1, inplace=True)


In [3]:
# # effect sizes # #
ds=pd.read_csv("/home/ls/rachelcw/projects/rare_disease/data/pilot1_ds_effect_sizes.txt",sep="\t",usecols=["intron","deltapsi"])
ds[['chrom', 'start', 'end','clu']] = ds['intron'].str.split(':', expand=True)
ds.drop(['intron'], axis=1, inplace=True)

In [6]:
table=ds.merge(cluster, on=["chrom","clu"], how="left")
# table.dropna(axis=0)

In [9]:
import pyensembl
# from pyensembl import EnsemblRelease
# data = EnsemblRelease(77)
# pyensembl install --release 77 --species homo_sapiens

data = pyensembl.Genome(
    reference_name='GRCh38',
    annotation_name='my_genome_lab',
    gtf_path_or_url='"/data01/private/resources/gencode.v34.GRCh38.annotation.gtf')
data.index()

# will return ['HLA-A']
# gene_names = data.gene_names_at_locus(contig=6, position=29945884)

MissingLocalFile: MissingFile(/data01/home/ls/rachelcw/projects/rare_disease/"/data01/private/resources/gencode.v34.GRCh38.annotation.gtf)

convert leafcutter results to bed format

In [5]:
cluster=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/lc_20230108/lc_20230108_perind_numers.counts.gz", sep=" ")

In [22]:
ds=pd.read_csv("/home/ls/rachelcw/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230115/ds.a1.20230115_effect_sizes.txt", sep="\t", usecols=["intron"])

In [23]:
ds[['chrom', 'start', 'end','clu']] = ds['intron'].str.split(':', expand=True)

# Drop columns
ds.drop(['intron','cluster'], axis=1, inplace=True)

# Write the BED file
ds.to_csv('/home/ls/rachelcw/projects/rare_disease/output_file.bed', sep='\t', index=False, header=False)